In [5]:
!pip install orjson

In [6]:
"""
Takes the CSV export of the `mal-user-animelists` table and prcesses it into the same format
as the original CSV ratings from Kaggle
"""

import csv
import sys
import pandas as pd
import orjson

csv.field_size_limit(sys.maxsize)

9223372036854775807

In [7]:
import csv, json

min_latest_updated = "2020-08-10T00:18:32+00:00"
min_rating_count = 10

all_anime_ids = set()
retained_user_count = 0
skipped_user_count = 0
total_retained_rating_count = 0

with open("./work/data/mal-user-animelists.csv", "rt", newline="") as rf, \
     open("./work/data/collected_animelists.csv", "wt", newline="") as wf:
    reader = csv.reader(rf)
    writer = csv.writer(wf)
    writer.writerow(["username", "anime_id", "my_score", "status", "start_date", "end_date"])

    # consume header once
    header = next(reader, None)

    for i, row in enumerate(reader, start=1):
        username = row[0]
        raw_json = row[1]

        if i % 10000 == 0:
            print(f"Processed {i} user profiles")

        try:
            parsed = parsed = orjson.loads(raw_json)
        except orjson.JSONDecodeError:
            skipped_user_count += 1
            continue

        if len(parsed) < min_rating_count:
            skipped_user_count += 1
            continue

        any_recent = False
        out_rows = []

        for node in parsed:
            ls = node.get("list_status") or {}
            updated_at = ls.get("updated_at")
            if (not any_recent) and updated_at and updated_at > min_latest_updated:
                any_recent = True

            score = ls.get("score") or 0
            # if score == 0:
            #     continue

            n = node.get("node") or {}
            anime_id = n.get("id")
            status = ls.get("status")
            if anime_id is None or status is None:
                continue

            all_anime_ids.add(anime_id)
            out_rows.append([
                username,
                anime_id,
                score,
                status,
                ls.get("start_date") or "",
                ls.get("end_date") or "",
            ])

        if not any_recent:
            skipped_user_count += 1
            continue

        if out_rows:
            writer.writerows(out_rows)

        total_retained_rating_count += len(parsed)
        retained_user_count += 1

print(f"Retained {retained_user_count} user profiles with {total_retained_rating_count} ratings")
print(f"Skipped {skipped_user_count} user profiles")


Processed 10000 user profiles
Processed 20000 user profiles
Processed 30000 user profiles
Processed 40000 user profiles
Processed 50000 user profiles
Processed 60000 user profiles
Processed 70000 user profiles
Processed 80000 user profiles
Processed 90000 user profiles
Processed 100000 user profiles
Processed 110000 user profiles
Processed 120000 user profiles
Processed 130000 user profiles
Processed 140000 user profiles
Processed 150000 user profiles
Processed 160000 user profiles
Processed 170000 user profiles
Processed 180000 user profiles
Processed 190000 user profiles
Processed 200000 user profiles
Processed 210000 user profiles
Processed 220000 user profiles
Processed 230000 user profiles
Processed 240000 user profiles
Processed 250000 user profiles
Processed 260000 user profiles
Processed 270000 user profiles
Processed 280000 user profiles
Processed 290000 user profiles
Processed 300000 user profiles
Processed 310000 user profiles
Processed 320000 user profiles
Processed 330000 

In [8]:
with open('./work/data/all-anime-ids.json', 'wt') as f:
    f.write(json.dumps(list(all_anime_ids)))